In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0

**Load model and processor**

In [6]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image

In [3]:
processor = DonutProcessor.from_pretrained("hoangphu7122002ai/donutAxis")
model = VisionEncoderDecoderModel.from_pretrained("hoangphu7122002ai/donutAxis")

In [7]:
image = Image.open('/content/0005e64fdc6e.jpg')

In [8]:
pixel_values = processor(image, return_tensors="pt").pixel_values
print(pixel_values.shape)

torch.Size([1, 3, 560, 560])


In [9]:
import torch

task_prompt = "<|PROMPT|>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

outputs = model.generate(pixel_values.to(device),
                               decoder_input_ids=decoder_input_ids.to(device),
                               max_length=model.decoder.config.max_position_embeddings,
                               early_stopping=True,
                               pad_token_id=processor.tokenizer.pad_token_id,
                               eos_token_id=processor.tokenizer.eos_token_id,
                               use_cache=True,
                               num_beams=1,
                               bad_words_ids=[[processor.tokenizer.unk_token_id]],
                               return_dict_in_generate=True,
                               output_scores=True,)

In [10]:
import re

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.decode([model.config.decoder_end_token_id]), "").replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
sequence = re.sub(r"<one>", "1", sequence)
sequence = re.sub(r"(?:(?<=>) | (?=</s_))", "", sequence)
sequence = sequence.split('<tick_label_y_end>')[0] + '<tick_label_y_end>'
print(sequence)

<|PROMPT|><chart_title_start>Rural population (%) long-run with 2050 projections (OWID) in Greece<chart_title_end><axis_title_x_start>Years<axis_title_x_end><axis_title_x_start>Percentage of Rural Population<axis_title_x_end><tick_label_x_start>1940;1960;1980;2000;2020;2040;2060<tick_label_x_end><tick_label_y_start>60;50;40;30;20;10;0<tick_label_y_end>


In [11]:
processor.token2json(sequence)

{'text_sequence': '<|PROMPT|><chart_title_start>Rural population (%) long-run with 2050 projections (OWID) in Greece<chart_title_end><axis_title_x_start>Years<axis_title_x_end><axis_title_x_start>Percentage of Rural Population<axis_title_x_end><tick_label_x_start>1940;1960;1980;2000;2020;2040;2060<tick_label_x_end><tick_label_y_start>60;50;40;30;20;10;0<tick_label_y_end>'}